# Exercise 3 - Oasis Keys Service

This exercise shows how to use the inbuilt keys service code to map exposure data to the model specific keys (area peril and vulnerability)

In [ ]:
# standard libraries
import json
import warnings

import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon

warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')

In [ ]:
# oed exposure data
df_loc = pd.read_csv('test_data/location.csv')

df_loc

In [ ]:
! oasislmf model generate-keys --help

In [ ]:
lookup_config = {
    "model": {
        "supplier_id": "OasisLMF",
        "model_id": "OasisHurdat",
        "model_version": "0.0.1"
    },
    "builtin_lookup_type": "new_lookup",
    "keys_data_path": "keys_data",
    "step_definition": {
        "area_peril":{
            "type": "rtree",
            "columns": ["latitude", "longitude"],
            "parameters": {
                "file_path": "keys_data/areaperil_dict.parquet",
                "file_type": "parquet",
                "id_columns": ["area_peril_id"], 
                "nearest_neighbor_min_distance": -1
            }
        },
        "split_loc_perils_covered":{
            "type": "split_loc_perils_covered" ,
            "columns": ["locperilscovered"],
            "parameters": {
                "model_perils_covered": ["WTC"]
            }
        },
        "create_coverage_type": {
            "type": "simple_pivot",
            "parameters": {
                "pivots": [{"new_cols": {"coverage_type": 1}}]
            }
        },
        "vulnerability": {
            "type": "merge",
            "columns": ["peril_id", "coverage_type", "occupancycode"],
            "parameters": {"file_path": "keys_data/vulnerability_dict.csv",
                           "id_columns": ["vulnerability_id"]
                          }
        }
    },
    "strategy": ["split_loc_perils_covered", "area_peril", "create_coverage_type", "vulnerability"]
}

lookup_config


In [ ]:
# write config file
with open('lookup_config.json','w') as lc:
    json.dump(lookup_config, lc)
    
! ls -ltr

In [ ]:
# generate areaperil grid 
df_grid = pd.read_csv('keys_data/areaperil_dict.csv')

df_grid['lon1'] = df_grid['longitude']-0.05
df_grid['lon2'] = df_grid['longitude']-0.05
df_grid['lon3'] = df_grid['longitude']+0.05
df_grid['lon4'] = df_grid['longitude']+0.05

df_grid['lat1'] = df_grid['latitude']-0.05
df_grid['lat2'] = df_grid['latitude']+0.05
df_grid['lat3'] = df_grid['latitude']-0.05
df_grid['lat4'] = df_grid['latitude']+0.05

df_grid['peril_id'] = 'WTC'

polygon_point_order = [1, 2, 4, 3]

gdf_peril_area = gpd.GeoDataFrame(df_grid)
gdf_peril_area["geometry"] = gdf_peril_area.apply(
    lambda row: Polygon([(row[f"lon{i}"], row[f"lat{i}"]) for i in polygon_point_order]), axis=1)


gfd_file_df = gdf_peril_area[['peril_id','area_peril_id','geometry']]

gfd_file_df.to_parquet('keys_data/areaperil_dict.parquet')

gfd_file_df


In [ ]:
with open('keys_data/vulnerability_dict.csv','w') as v:
    v.write('PERIL_ID,COVERAGE_TYPE,OCCUPANCYCODE,VULNERABILITY_ID\n')
    v.write('WTC,1,1101,1')
    
! cat keys_data/vulnerability_dict.csv 

In [ ]:
! oasislmf model generate-keys -x 'test_data/location.csv' -g 'lookup_config.json' -d 'keys_data'

In [ ]:
! head <<<'copy in "Keys successful" path from above message'>>>